Copyright **Paolo Raiteri**, January 2022

# Numerical solution of equilibrium problems

Let's consider a generic chemical reaction 

\begin{equation}
aA + bB + \dots \leftrightharpoons xX + yY + \dots
\end{equation}

if the concentrations of the species are not the equilibrium ones, the reaction will progress (either forward or backwards) to reach the equilibrium conditions, and the free energy of the system will decrease.

An infinitesimal change of the system's free energy can then be written in terms of the chemical potential, $\mu$, of all the species involved in the reaction.

\begin{equation}
\mathrm{d}G = \sum_i \mu_i \mathrm{d}n_i
\end{equation}

Because the concentrations of reactants and products are coupled through the reaction equation, they cannot change independently, _i.e._ if any products are formed, some reactants must be consumed.
We can therefore define a unique parameter, $\xi$, as the _progress of the reaction_ 

\begin{equation}
\mathrm{d}\xi = \frac{\mathrm{d}n_i}{ \nu_i }
\end{equation}

where $\nu_i$ is the stoichiometric coefficient of the species $i$ taken as a positive number for products (formed) and as a negative number for reactants (consumed), and $\mathrm{d}n_i$ is an infinitesimal change in the concentration of species $i$.
Thus, the free energy change due to the progress of the reaction (in any direction) becomes

\begin{equation}
\mathrm{d}G = \sum_i \nu_i \mu_i \mathrm{d}\xi
\end{equation}

At equilibrium, the concentrations of all species are constant, $\mathrm{d}n_i=0$, and the reaction is not _progressing_ anymore, $\mathrm{d}\xi=0$, hence the free energy is not changing anymore.
On the other hand, if the system is out of equilibrium, there is a driving force that pushes the system towards equilibrium. 
A force can always be obtained as the negative of the derivative of the energy with respect to some coordinate.
In this case, the driving force that pushes the reaction towards equilibrium can be written as

\begin{equation}
\frac{\mathrm{d}G}{\mathrm{d}\xi} = \sum_i \nu_i \mu_i = -force
\end{equation}

If we then substitute the definition of the chemical potential, $\mu=\mu^0+RT\ln x$, where $x$ is the molar fraction of the species, we obtain

\begin{eqnarray}
\frac{\mathrm{d} G}{\mathrm{d}\xi} &=& \sum_i\nu_i\mu_i\\
&=& \sum_i\nu_i \big[\mu_i^0 +RT\ln x_i\big]\\
&=& \sum_i\nu_i\mu_i^0 +RT\sum_i\nu_i\ln x_i\\
&=& \Delta_r G^0 +RT\sum_i\ln x_i^{\nu_i}\\
&=& \Delta_r G^0 +RT\ln \prod_i x_i^{\nu_i}\\
&=& \Delta_r G^0 +RT\ln Q\\
\end{eqnarray}

here $Q$ is the reaction quotient. By remembering the definition of equilibrium constant, $-RT\ln K_{eq}=\Delta_r G^0$ we get

\begin{equation}
force = -\frac{\mathrm{d} G}{\mathrm{d}\xi} = RT\ln K_{eq} - RT\ln Q = RT\ln\ \frac{K_{eq}}{Q}
\end{equation}

This equation provides us with a conceptual framework to solve *numerically* any equilibrium problem.
Because the above equations are differential, they are valid only for infinitesimal changes in the conditions.
Hence each a small change in the concentrations, $\nu_i\mathrm{d}c$ the driving force may change significantly.

Therefore we need to set up an iterative procedure where we compute the driving force, change the concentrations, compute the driving force\dots until the equilibrium is reached.

Given the concentration of all species, we can compute the driving force using the equation above and alter the concentrations of all species using

\begin{equation}
\mathrm{d}n_i \approx \nu_i \times force \times \mathrm{d}c 
\end{equation}

where $\nu_i$ are the stoichiometric coefficients of the species and $\mathrm{d}c$ is an adjustable parameter.
$\mathrm{d}c$ must be small enough to maintain the validity of the approximation abover but not too small to allow for a quick convergence of the procedure.


## Example: Dimerisation reaction
In order to elucidate how that procedure works we can take a model dimerisation reaction

\begin{equation}
2A \leftrightharpoons B
\end{equation}

whose equilibrium constant can be written as
\begin{equation}
K_{eq} = \frac{[B]}{[A]^2} = 0.156
\end{equation}

We now want to calculate the equilibrium concentrations of $[A]_{eq}$ and $[B]_{eq}$ given their initial concentrations $[A]_{0}$ and $[B]_{0}$.
Although this is a simple problem that can be solved analytically, in this workshop we will learn how we can use an iterative method to numerically solve it.
We will use a relatively simple minimisation procedure that can be applied to a large number of problems, for which it is not possible or it is too complicated to get an analytic solution.

Imagine mixing the reagents and then to be able to monitor the concentration of all the species in the system at very short discrete time intervals (*timesteps*). What you will see is that the concentrations will change and tend to the equilibrium value. As you have learnt in first year, the reaction quotient, $Q$, can be used to decided which way to reaction will proceed, and that at equilibrium the reaction quotient is equal to the equilibrium constant. Hence, as we have discussed in class, the reaction quotient and the equilibrium constant can be use to define a *driving force* that pulls the system towards equilibrium.

This *driving force* can then be used in conjunction with an *ICE* table to numerically compute the equilibrium concentration of reactant and products.


The working principle of the minimisation procedure that we will employ is

1. compute the reaction quotient
\begin{equation}
Q = \dfrac{\mathrm{[B]}_0}{\mathrm{[A]}^2_0}
\end{equation}

2. compute the driving force

\begin{equation}
F = RT \ln\bigg[\frac{K_{eq}}{Q}\bigg]
\end{equation}

3. evolve the concentrations using the ICE table

|    <img width=50/>  | [A]  <img width=150/>      | [B]<img width=150/>
| :---: | :--------: |:---------:
| *I*  | [A]$_0$    | [B]$_0$
| *C*  | -2$F\delta c$        | +$F\delta c$
| *E*  | [A]$_0$-2$F\delta c$ | [B]$_0$+$F\delta c$

4. Repeat until the solution does not change 

In [ ]:
# python packages
from sys import stdout
import pandas as pd # allows us to use dataframes
import numpy as np
import math
import matplotlib.pyplot as plt # allows us to make graphs
# below prevents warnings from outdated Pandas append command clogging output
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
%matplotlib notebook

In [ ]:
# Cell below fixes scrolling issues for long outputs

In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
   return false;
}

In [ ]:
# initial conditions
RT = 8.314 * 298.15
Keq = 0..561

# adjustable parameter dc
dc = 1e-7

# initial concentrations and stoichiometric coefficient
concA = 0.15 ; nuA = -1
concB = 0.05 ; nuB = 1

# create a data frame for plotting
Data = {'Cycle': [0],
        '[A]': [concA],
        '[B]': [concB],
       }
df = pd.DataFrame(Data, columns=['Cycle','[A]','[B]'])

# print the data header
print("{:<30}{:<12}{:<12}{:<12}{:<12}".format("","[A]","[B]", "Q", "Keq"))

# reaction quotient ...
Q = concB / (concA*concA)
# print the first set of data
print("{:<30}{:<12}{:<12}{:<12}{:<12}".format("FIRST REACTION QUOTIENT","{:.5f}".format(concA),"{:.5f}".format(concB), "{:.5f}".format(Q), Keq))


### Critical thinking questions
1. Verify that the final equilibrium concentrations are independent of the starting conditions, provided that $[A]+[B]$ is constant
2. What is the effect of changing the values of $dc$ or $nCycles$?
3. Is the value of RT important?

## Example dissociation of mono-protic acid
As an exercise now try to calculate the final pH of a solution of a monoprotic acid, _e.g._ acetic acid, $pK_a=4.756$.

The equilibrium constant for the reation

\begin{equation}
AH \leftrightharpoons A^- + H^+
\end{equation} 

is $K_{eq}=10^{-4.756}=1.75\times10^{-5}$.

1. Determine the pH of a solution where $[AH]+[A^-]$=0.15M
2. What is the concentration of $[AH]$ if the pH is maintained at a value of 6 by external titration?